## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Pacific Grove,US,36.6177,-121.9166,61.56,74,0,4.00,clear sky
1,1,Cape Town,ZA,-33.9258,18.4232,77.11,36,0,17.27,clear sky
2,2,Iqaluit,CA,63.7506,-68.5145,20.93,86,100,10.36,light snow
3,3,Hobart,AU,-42.8794,147.3294,55.06,87,20,6.91,few clouds
4,4,Ibipeba,BR,-11.6408,-42.0111,83.46,35,5,8.03,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Cape Town,ZA,-33.9258,18.4232,77.11,36,0,17.27,clear sky
4,4,Ibipeba,BR,-11.6408,-42.0111,83.46,35,5,8.03,clear sky
8,8,Port Hedland,AU,-20.3167,118.5667,84.25,79,9,11.50,clear sky
12,12,Vaini,TO,-21.2000,-175.2000,73.56,100,20,2.30,few clouds
13,13,Albina,SR,5.5000,-54.0500,78.22,92,100,5.23,overcast clouds
22,22,Vila Velha,BR,-20.3297,-40.2925,84.15,72,75,9.22,broken clouds
32,32,Ballina,AU,-28.8667,153.5667,73.36,81,40,10.36,scattered clouds
36,36,Hithadhoo,MV,-0.6000,73.0833,83.62,64,57,7.94,broken clouds
37,37,Arraial Do Cabo,BR,-22.9661,-42.0278,80.56,73,20,11.50,few clouds
40,40,Bosaso,SO,11.2842,49.1816,83.93,62,98,8.21,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df=preferred_cities_df.dropna()

In [7]:
clean_preferred_cities_df.count()

City_ID                274
City                   274
Country                274
Lat                    274
Lng                    274
Max Temp               274
Humidity               274
Cloudiness             274
Wind Speed             274
Weather Description    274
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,77.11,clear sky,-33.9258,18.4232,
4,Ibipeba,BR,83.46,clear sky,-11.6408,-42.0111,
8,Port Hedland,AU,84.25,clear sky,-20.3167,118.5667,
12,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,
13,Albina,SR,78.22,overcast clouds,5.5000,-54.0500,
22,Vila Velha,BR,84.15,broken clouds,-20.3297,-40.2925,
32,Ballina,AU,73.36,scattered clouds,-28.8667,153.5667,
36,Hithadhoo,MV,83.62,broken clouds,-0.6000,73.0833,
37,Arraial Do Cabo,BR,80.56,few clouds,-22.9661,-42.0278,
40,Bosaso,SO,83.93,overcast clouds,11.2842,49.1816,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        pass
    
#Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
   
      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,77.11,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
4,Ibipeba,BR,83.46,clear sky,-11.6408,-42.0111,Quitanda santos
8,Port Hedland,AU,84.25,clear sky,-20.3167,118.5667,The Esplanade Hotel
12,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
13,Albina,SR,78.22,overcast clouds,5.5000,-54.0500,Hudace
22,Vila Velha,BR,84.15,broken clouds,-20.3297,-40.2925,Hotel Vitória Palace
32,Ballina,AU,73.36,scattered clouds,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron
36,Hithadhoo,MV,83.62,broken clouds,-0.6000,73.0833,Scoop Guest House
37,Arraial Do Cabo,BR,80.56,few clouds,-22.9661,-42.0278,Pousada Porto Praia
40,Bosaso,SO,83.93,overcast clouds,11.2842,49.1816,Red Sea Hotel


In [11]:
hotel_df.isnull().sum() 

City                   0
Country                0
Max Temp               0
Weather Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [12]:
# 7.Drop the rows where there is no Hotel Name.(Actually don't have any null value)
clean_hotel_data_df=hotel_df.dropna()
clean_hotel_data_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,77.11,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
4,Ibipeba,BR,83.46,clear sky,-11.6408,-42.0111,Quitanda santos
8,Port Hedland,AU,84.25,clear sky,-20.3167,118.5667,The Esplanade Hotel
12,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
13,Albina,SR,78.22,overcast clouds,5.5000,-54.0500,Hudace
22,Vila Velha,BR,84.15,broken clouds,-20.3297,-40.2925,Hotel Vitória Palace
32,Ballina,AU,73.36,scattered clouds,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron
36,Hithadhoo,MV,83.62,broken clouds,-0.6000,73.0833,Scoop Guest House
37,Arraial Do Cabo,BR,80.56,few clouds,-22.9661,-42.0278,Pousada Porto Praia
40,Bosaso,SO,83.93,overcast clouds,11.2842,49.1816,Red Sea Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_data_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_data_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_data_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))